<p><b><center><span style="font-size:26px;" </span> <span style="font-family=Times new roman;"</span> Understanding different Mean Encodings schemes & Implementing them in Python</center></b></p>

<p><center><span style="font-size:16px;"</span> By Abdul Muqtadir()</center></p>


# 1. Introduction
Mean Encoding
1. In this tutorial we are going to Mean encode "Embarked" column. Using four different encoding schemes.
    * Leave one out 
    * Smoothing
    * Expanding mean
2. Then we are going to find corelation in between every encoding scheme and target.

In [1]:
# Importing libraries
import pandas as pd
import numpy as np

# Reading Dataset
Train_Data = pd.read_csv("../input/simple-dataset-for-easy-practice-for-beginners/BrandEDA (1).csv")

In [2]:
# About Dataset

#Train_Data.shape
#Train_Data.info()
#Train_Data.dtypes
#Train_Data.nunique()
#Train_Data["Item ID"].unique()

Train_Data.head()


,Brand,Item ID,Ram(Gb),StoraGe(Gb),Price(Rs),Processor speed(GHz),Touch,Color,Weight,DisplAy size(Inch)
0,Apple,32,16.0,256.0,315000,2.2,No,Space grey,1.83,15.4
1,Apple,8,8.0,512.0,232000,3.2,No,Dark Black,1.29,13.3
2,Apple,32,16.0,1024.0,382000,2.3,No,Silver,2.00,16.0
3,Apple,26,8.0,NaN,195000,2.3,No,Space grey,1.37,13.3
4,Apple,32,4.0,128.0,92400,1.8,No,Silver,1.35,11.6


In [3]:
#Drop multiple columns (Just to make dataset bit easier for demonstration)
#Train_Data.drop(Train_Data.iloc[:, 14:26], axis=1, inplace=True)

In [4]:
Train_Data.sort_values(['Item ID'],inplace=True)

In [5]:
Train_Data["Brand"].replace({"Apple":"0", "Dell":"1"}, inplace = True)
Train_Data = Train_Data.astype({"Brand": int})

In [6]:
Train_Data.head()
IID_GMean = Train_Data['Item ID'].mean()

# Leave one out scheme

In [7]:
# Calculate sum of the target values using all the objects.
target_sum = Train_Data.groupby('Item ID')['Brand'].transform('sum')

# Then subtract the target of the given object and divide the resulting value by n_objects - 1.
n_objects = Train_Data.groupby('Item ID')['Brand'].transform('count')


Train_Data['item_target_enc'] = (target_sum - Train_Data['Brand']) / (n_objects - 1)
Train_Data['item_target_enc'].fillna(0.3343, inplace=True)


In [8]:
target_sum

9     3
16    3
26    3
25    3
5     3
7     3
23    3
21    7
28    7
13    7
12    7
14    7
10    7
27    7
1     7
22    7
18    7
20    3
24    3
29    3
17    3
8     3
3     3
19    3
15    2
11    2
6     2
4     2
2     2
0     2
Name: Brand, dtype: int64

In [9]:
Train_Data.shape
Train_Data.head(30)

,Brand,Item ID,Ram(Gb),StoraGe(Gb),Price(Rs),Processor speed(GHz),Touch,Color,Weight,DisplAy size(Inch),item_target_enc
9,0,4,16.0,NaN,382000,2.60,No,Silver,2.00,16.0,0.500000
16,1,4,8.0,256.0,185000,1.80,No,Silver,1.23,13.3,0.333333
26,1,4,4.0,250.0,25000,2.35,No,Black,1.63,15.6,0.333333
25,1,4,8.0,1024.0,170000,1.60,No,Black,2.20,15.6,0.333333
5,0,4,8.0,256.0,60000,2.60,No,Silver,1.00,13.3,0.500000
7,0,4,8.0,128.0,224700,1.80,No,Silver,1.23,13.3,0.500000
23,0,4,8.0,256.0,255000,2.30,No,Silver,1.37,13.3,0.500000
21,0,8,4.0,128.0,101000,1.80,No,Silver,NaN,11.0,0.777778
28,1,8,6.0,650.0,33200,2.26,No,White,2.84,15.6,0.666667
13,1,8,NaN,1024.0,125000,1.80,Yes,Silver,1.90,15.6,0.666667


In [10]:
%%time

# YOUR CODE GOES HERE
alpha = 100

item_id_target_mean = Train_Data.groupby('Item ID')['Brand'].transform('mean')
n_objects = Train_Data.groupby('Item ID')['Brand'].transform('count')

Train_Data['item_target_enc'] = (item_id_target_mean * n_objects + 0.3343*alpha) / (n_objects + alpha)

Train_Data['item_target_enc'].fillna(0.3343, inplace=True)

CPU times: user 2.5 ms, sys: 966 µs, total: 3.47 ms
Wall time: 4.61 ms


In [11]:
Train_Data.shape
Train_Data.head(30)

,Brand,Item ID,Ram(Gb),StoraGe(Gb),Price(Rs),Processor speed(GHz),Touch,Color,Weight,DisplAy size(Inch),item_target_enc
9,0,4,16.0,NaN,382000,2.60,No,Silver,2.00,16.0,0.340467
16,1,4,8.0,256.0,185000,1.80,No,Silver,1.23,13.3,0.340467
26,1,4,4.0,250.0,25000,2.35,No,Black,1.63,15.6,0.340467
25,1,4,8.0,1024.0,170000,1.60,No,Black,2.20,15.6,0.340467
5,0,4,8.0,256.0,60000,2.60,No,Silver,1.00,13.3,0.340467
7,0,4,8.0,128.0,224700,1.80,No,Silver,1.23,13.3,0.340467
23,0,4,8.0,256.0,255000,2.30,No,Silver,1.37,13.3,0.340467
21,0,8,4.0,128.0,101000,1.80,No,Silver,NaN,11.0,0.367545
28,1,8,6.0,650.0,33200,2.26,No,White,2.84,15.6,0.367545
13,1,8,NaN,1024.0,125000,1.80,Yes,Silver,1.90,15.6,0.367545
